# Topic Model Evaluation

This notebook contains code needed to run the experiments of my Bachelor's thesis on Topic Modeling Algorithms.

For installation please check out `README.md` of this repository if you haven't already.

In [1]:
import sys
sys.path.append('../')

In [2]:
# Import neccessary packages
import numpy as np
from sentence_transformers import SentenceTransformer

from evaluation import DataLoader, Trainer

/home/budu/BERTopic_evaluation/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/budu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Trump's tweets

First we will run BERTopic and Top2Vec on Donald Trump's tweets.

His tweets' archive can be found here: https://www.thetrumparchive.com/

For our experiments we will not consider retweets and deleted tweets, those are being filtered out during the operation of `DataLoader` class.

In [3]:
dataset, custom = "trump", True

In [4]:
%%time
dataloader_trump = DataLoader(dataset=dataset).\
                    prepare_docs(save=f"{dataset}.txt").\
                    preprocess_octis(output_folder=f"{dataset}")

100%|█████████████████████████████████████████████████████████████████████████| 46693/46693 [00:00<00:00, 253384.77it/s]


created vocab
53637
words filtering done
CPU times: user 2.62 s, sys: 284 ms, total: 2.9 s
Wall time: 12.8 s


To speed up BERTopic we precalculate embeddings using `SentenceTransformer all-mpnet-base-v2` model. Otherwise, in every different parameter setting would require calculating embeddings, and that would result in a massive runtime.

In [6]:
# Prepare data
dataset, custom = "trump", True
dataloader = DataLoader(dataset)
data_trump = dataloader.load_octis(custom)
data_trump = [" ".join(words) for words in data_trump.get_corpus()]

# Extract embeddings
model = SentenceTransformer("all-mpnet-base-v2")
embeddings_trump = model.encode(data_trump, show_progress_bar=True)

Batches: 100%|██████████████████████████████████████████████████████████████████████| 1383/1383 [13:28<00:00,  1.71it/s]


In [7]:
# Save embeddings
np.savetxt('embeddings_trump.txt', embeddings_trump)

As described in the Thesis and the BERTopic paper [https://arxiv.org/pdf/2203.05794.pdf] the performance of the topic models is reflected by two widely-used metrics, topic coherence and topic diversity.

For each topic model, coherence is evaluated using Normalized Pointwise Mutual Information (NPMI). This measure ranges from $-1$ to $1$, where $1$ indicates a perfect association.

Topic diversity is the percentage of unique words for all topics. This measure ranges from $0$ to $1$ where $0$ indicates redundant topic and $1$ indicates varied topic.

Ranging from $10$ to $50$ topics with steps of $10$, the NPMI score is calculated at each step for each topic model. Results are averaged across $3$ runs for each step.

In [6]:
# Evaluating BERTopic on Trump's tweets

for i in range(3):

    params = {
        "embedding_model": "all-mpnet-base-v2",
        "nr_topics": [(i+1)*10 for i in range(5)],
        "min_topic_size": 15,
        "verbose": True
    }

    trainer = Trainer(dataset=dataset,
                      model_name="BERTopic",
                      params=params,
                      bt_embeddings=embeddings_trump,
                      custom_dataset=custom,
                      verbose=True)
    results = trainer.train(save=f"../results/Basic/Trump/bertopic_{i+1}")

2024-04-29 17:29:12,850 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-29 17:30:43,513 - BERTopic - Dimensionality - Completed ✓
2024-04-29 17:30:43,517 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-29 17:30:47,714 - BERTopic - Cluster - Completed ✓
2024-04-29 17:30:47,717 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-29 17:30:49,622 - BERTopic - Representation - Completed ✓
2024-04-29 17:30:49,625 - BERTopic - Topic reduction - Reducing number of topics
2024-04-29 17:30:51,283 - BERTopic - Topic reduction - Reduced number of topics from 379 to 10


Results
npmi: -0.055941250571217155
diversity: 0.7222222222222222
 


2024-04-29 17:31:07,815 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-29 17:31:45,255 - BERTopic - Dimensionality - Completed ✓
2024-04-29 17:31:45,258 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-29 17:31:48,480 - BERTopic - Cluster - Completed ✓
2024-04-29 17:31:48,482 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-29 17:31:50,341 - BERTopic - Representation - Completed ✓
2024-04-29 17:31:50,343 - BERTopic - Topic reduction - Reducing number of topics
2024-04-29 17:31:51,821 - BERTopic - Topic reduction - Reduced number of topics from 368 to 20


Results
npmi: -0.045564519617954206
diversity: 0.7105263157894737
 


2024-04-29 17:32:10,286 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-29 17:32:51,209 - BERTopic - Dimensionality - Completed ✓
2024-04-29 17:32:51,213 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-29 17:32:54,290 - BERTopic - Cluster - Completed ✓
2024-04-29 17:32:54,292 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-29 17:32:56,356 - BERTopic - Representation - Completed ✓
2024-04-29 17:32:56,362 - BERTopic - Topic reduction - Reducing number of topics
2024-04-29 17:32:57,772 - BERTopic - Topic reduction - Reduced number of topics from 380 to 30


...

Results
npmi: -0.057880019457847015
diversity: 0.7724137931034483
 


2024-04-29 17:43:07,143 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-29 17:43:40,301 - BERTopic - Dimensionality - Completed ✓
2024-04-29 17:43:40,304 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-29 17:43:43,400 - BERTopic - Cluster - Completed ✓
2024-04-29 17:43:43,402 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-29 17:43:45,171 - BERTopic - Representation - Completed ✓
2024-04-29 17:43:45,173 - BERTopic - Topic reduction - Reducing number of topics
2024-04-29 17:43:46,671 - BERTopic - Topic reduction - Reduced number of topics from 372 to 40


Results
npmi: -0.02733355584221016
diversity: 0.7769230769230769
 


2024-04-29 17:44:06,543 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-29 17:44:39,834 - BERTopic - Dimensionality - Completed ✓
2024-04-29 17:44:39,838 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-29 17:44:43,185 - BERTopic - Cluster - Completed ✓
2024-04-29 17:44:43,189 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-29 17:44:45,034 - BERTopic - Representation - Completed ✓
2024-04-29 17:44:45,036 - BERTopic - Topic reduction - Reducing number of topics
2024-04-29 17:44:46,590 - BERTopic - Topic reduction - Reduced number of topics from 374 to 50


Results
npmi: -0.052933953618729726
diversity: 0.8122448979591836
 


In [9]:
# Evaluating Top2Vec on Trump's tweets

for i in range(3):

    params = {"nr_topics": [(i+1)*10 for i in range(5)],
              "hdbscan_args": {'min_cluster_size': 15,
                               'metric': 'euclidean',
                               'cluster_selection_method': 'eom'}}

    trainer = Trainer(dataset=dataset,
                      custom_dataset=custom,
                      custom_model=None,
                      model_name="Top2Vec",
                      params=params,
                      verbose=True)
    results = trainer.train(save=f"../results/Basic/Trump/Top2Vec_{i+1}")

2024-04-29 09:51:40,302 - top2vec - INFO - Pre-processing documents for training
2024-04-29 09:51:41,683 - top2vec - INFO - Creating joint document/word embedding
2024-04-29 09:56:17,586 - top2vec - INFO - Creating lower dimension embedding of documents
2024-04-29 09:56:30,994 - top2vec - INFO - Finding dense areas of documents
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers

Results
npmi: -0.10839552498307084
diversity: 0.73
 


2024-04-29 09:57:04,728 - top2vec - INFO - Creating joint document/word embedding
2024-04-29 10:01:50,596 - top2vec - INFO - Creating lower dimension embedding of documents
2024-04-29 10:02:04,539 - top2vec - INFO - Finding dense areas of documents
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. D

Results
npmi: -0.1564537046781506
diversity: 0.7
 


2024-04-29 10:02:37,956 - top2vec - INFO - Creating joint document/word embedding
2024-04-29 10:07:20,156 - top2vec - INFO - Creating lower dimension embedding of documents
2024-04-29 10:07:37,740 - top2vec - INFO - Finding dense areas of documents
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. D

...

Results
npmi: -0.13877093061024934
diversity: 0.62
 


2024-04-29 11:03:18,383 - top2vec - INFO - Creating joint document/word embedding
2024-04-29 11:07:52,911 - top2vec - INFO - Creating lower dimension embedding of documents
2024-04-29 11:08:06,787 - top2vec - INFO - Finding dense areas of documents
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. D

Results
npmi: -0.1794100842662193
diversity: 0.6625
 


2024-04-29 11:08:36,539 - top2vec - INFO - Creating joint document/word embedding
2024-04-29 11:13:16,493 - top2vec - INFO - Creating lower dimension embedding of documents
2024-04-29 11:13:33,125 - top2vec - INFO - Finding dense areas of documents
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. D

Results
npmi: -0.20866024629079075
diversity: 0.678
 


## 20Newsgroups

The 20Newsgroups dataset comprises 16309 newsgroups posts on 20 topics.

In [7]:
dataset, custom = "20NewsGroup", False
dataloader_20ng = DataLoader(dataset)
data_20ng = dataloader_20ng.load_octis(custom)
data_20ng = [" ".join(words) for words in data_20ng.get_corpus()]

# Extract embeddings
#model = SentenceTransformer("all-mpnet-base-v2")
#embeddings_20ng = model.encode(data_20ng, show_progress_bar=True)

In [11]:
# Save embeddings
np.savetxt('embeddings_20ng.txt', embeddings_20ng)

In [12]:
# Evaluating BERTopic on 20Newsgroups

for i in range(3):

    params = {
        "embedding_model": "all-mpnet-base-v2",
        "nr_topics": [(i+1)*10 for i in range(5)],
        "min_topic_size": 15,
        "verbose": True
    }

    trainer = Trainer(dataset=dataset,
                      model_name="BERTopic",
                      params=params,
                      bt_embeddings=embeddings_20ng,
                      custom_dataset=custom,
                      verbose=True)
    results = trainer.train(save=f"../results/Basic/20NewsGroup/bertopic_{i+1}")

2024-04-29 11:49:36,292 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-29 11:49:45,430 - BERTopic - Dimensionality - Completed ✓
2024-04-29 11:49:45,432 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-29 11:49:45,878 - BERTopic - Cluster - Completed ✓
2024-04-29 11:49:45,879 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-29 11:49:46,511 - BERTopic - Representation - Completed ✓
2024-04-29 11:49:46,512 - BERTopic - Topic reduction - Reducing number of topics
2024-04-29 11:49:47,093 - BERTopic - Topic reduction - Reduced number of topics from 90 to 10


Results
npmi: 0.099337322297008
diversity: 0.9111111111111111
 


2024-04-29 11:49:54,140 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-29 11:50:03,935 - BERTopic - Dimensionality - Completed ✓
2024-04-29 11:50:03,936 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-29 11:50:04,371 - BERTopic - Cluster - Completed ✓
2024-04-29 11:50:04,372 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-29 11:50:04,966 - BERTopic - Representation - Completed ✓
2024-04-29 11:50:04,967 - BERTopic - Topic reduction - Reducing number of topics
2024-04-29 11:50:05,555 - BERTopic - Topic reduction - Reduced number of topics from 75 to 20


Results
npmi: 0.11307398932436573
diversity: 0.7684210526315789
 


2024-04-29 11:50:13,484 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-29 11:50:20,309 - BERTopic - Dimensionality - Completed ✓
2024-04-29 11:50:20,311 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-29 11:50:20,753 - BERTopic - Cluster - Completed ✓
2024-04-29 11:50:20,754 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-29 11:50:21,332 - BERTopic - Representation - Completed ✓
2024-04-29 11:50:21,333 - BERTopic - Topic reduction - Reducing number of topics
2024-04-29 11:50:21,917 - BERTopic - Topic reduction - Reduced number of topics from 84 to 30


...

Results
npmi: 0.12545060235506214
diversity: 0.8103448275862069
 


2024-04-29 11:53:43,282 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-29 11:53:50,190 - BERTopic - Dimensionality - Completed ✓
2024-04-29 11:53:50,192 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-29 11:53:50,631 - BERTopic - Cluster - Completed ✓
2024-04-29 11:53:50,633 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-29 11:53:51,295 - BERTopic - Representation - Completed ✓
2024-04-29 11:53:51,295 - BERTopic - Topic reduction - Reducing number of topics
2024-04-29 11:53:51,873 - BERTopic - Topic reduction - Reduced number of topics from 84 to 40


Results
npmi: 0.11900921035347693
diversity: 0.7897435897435897
 


2024-04-29 11:54:01,181 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-29 11:54:08,962 - BERTopic - Dimensionality - Completed ✓
2024-04-29 11:54:08,963 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-29 11:54:09,408 - BERTopic - Cluster - Completed ✓
2024-04-29 11:54:09,410 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-29 11:54:10,004 - BERTopic - Representation - Completed ✓
2024-04-29 11:54:10,005 - BERTopic - Topic reduction - Reducing number of topics
2024-04-29 11:54:10,552 - BERTopic - Topic reduction - Reduced number of topics from 88 to 50


Results
npmi: 0.11900244987039556
diversity: 0.7551020408163265
 


In [ ]:
# Evaluating Top2Vec on 20NewsGroups

for i in range(3):

    params = {"nr_topics": [(i+1)*10 for i in range(5)],
              "hdbscan_args": {'min_cluster_size': 15,
                               'metric': 'euclidean',
                               'cluster_selection_method': 'eom'}}

    trainer = Trainer(dataset=dataset,
                      custom_dataset=custom,
                      custom_model=None,
                      model_name="Top2Vec",
                      params=params,
                      verbose=True)
    results = trainer.train(save=f"../results/Basic/20NewsGroup/Top2Vec_{i+1}")

2024-04-29 17:54:13,368 - top2vec - INFO - Pre-processing documents for training
2024-04-29 17:54:16,742 - top2vec - INFO - Creating joint document/word embedding
2024-04-29 17:58:09,316 - top2vec - INFO - Creating lower dimension embedding of documents
2024-04-29 17:58:21,903 - top2vec - INFO - Finding dense areas of documents
2024-04-29 17:58:22,808 - top2vec - INFO - Finding topics
2024-04-29 17:58:48,238 - top2vec - INFO - Pre-processing documents for training


Results
npmi: 0.19436396186822624
diversity: 0.97
 


2024-04-29 17:58:51,167 - top2vec - INFO - Creating joint document/word embedding
2024-04-29 18:02:48,505 - top2vec - INFO - Creating lower dimension embedding of documents
2024-04-29 18:03:01,720 - top2vec - INFO - Finding dense areas of documents
2024-04-29 18:03:02,612 - top2vec - INFO - Finding topics
2024-04-29 18:03:31,644 - top2vec - INFO - Pre-processing documents for training


Results
npmi: 0.20609795474080722
diversity: 0.88
 


2024-04-29 18:03:34,631 - top2vec - INFO - Creating joint document/word embedding
2024-04-29 18:07:33,930 - top2vec - INFO - Creating lower dimension embedding of documents
2024-04-29 18:07:46,880 - top2vec - INFO - Finding dense areas of documents
2024-04-29 18:07:47,762 - top2vec - INFO - Finding topics
2024-04-29 18:08:14,824 - top2vec - INFO - Pre-processing documents for training


...

2024-04-29 18:46:22,913 - top2vec - INFO - Creating joint document/word embedding
2024-04-29 18:47:49,105 - top2vec - INFO - Creating lower dimension embedding of documents
2024-04-29 18:47:54,801 - top2vec - INFO - Finding dense areas of documents
2024-04-29 18:47:55,163 - top2vec - INFO - Finding topics
2024-04-29 18:48:05,677 - top2vec - INFO - Pre-processing documents for training


Results
npmi: 0.18974979893393054
diversity: 0.8333333333333334
 


2024-04-29 18:48:07,331 - top2vec - INFO - Creating joint document/word embedding
2024-04-29 18:49:43,819 - top2vec - INFO - Creating lower dimension embedding of documents
2024-04-29 18:49:49,711 - top2vec - INFO - Finding dense areas of documents
2024-04-29 18:49:50,110 - top2vec - INFO - Finding topics
2024-04-29 18:50:01,922 - top2vec - INFO - Pre-processing documents for training


Results
npmi: 0.1821333753194903
diversity: 0.7275
 


2024-04-29 18:50:03,116 - top2vec - INFO - Creating joint document/word embedding
2024-04-29 18:51:32,535 - top2vec - INFO - Creating lower dimension embedding of documents
2024-04-29 18:51:40,265 - top2vec - INFO - Finding dense areas of documents
2024-04-29 18:51:40,703 - top2vec - INFO - Finding topics
